# Group existing generators within regions and create new resources

This notebook shows how to use the PowerGenome API to cluster existing resources and create new-build resource options by model region. You'll need a settings file, such as the `test_settings.yml` provided in the `example_system` folder, with the following parameters.

For both existing and new resource:
- model_regions
- region_aggregations
- model_year
- target_usd_year
- atb_usd_year
- startup_fuel_use
- startup_vom_costs_mw
- startup_vom_costs_usd_year
- startup_costs_type
- startup_costs_per_cold_start_mw
- startup_costs_per_cold_start_usd_year
- existing_startup_costs_tech_map
- new_build_startup_costs

Specific to existing resource:
- num_clusters
- retirement_ages
- atb_existing_year
- existing_om_muiltiplier
- eia_atb_tech_map
- proposed_status_included
- proposed_gen_heat_rates
- proposed_min_load

Specific to new-build resources:
- atb_cost_case
- atb_financial_case
- atb_cap_recovery_years
- atb_new_gen
- renewables_clusters
- cost_multiplier_region_map
- cost_multiplier_technology_map
- transmission_investment_cost (if spur-lines are needed for non-renewable resources)

To calculate fuel costs for each resource:
- aeo_fuel_region_map
- eia_series_region_names
- eia_series_fuel_names
- eia_aeo_year
- eia_series_scenario_names
- aeo_fuel_scenarios
- aeo_fuel_usd_year
- tech_fuel_map
- fuel_emission_factors
- carbon_tax (optional)

And if CCS resources are included:
- ccs_fuel_map
- ccs_capture_rate
- ccs_disposal_cost

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from pathlib import Path

import geopandas as gpd
import numpy as np
import pandas as pd
from powergenome.generators import GeneratorClusters
from powergenome.GenX import reduce_time_domain
from powergenome.load_profiles import make_final_load_curves
from powergenome.params import DATA_PATHS
from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
    check_settings
)
from powergenome.external_data import (
    make_demand_response_profiles,
    make_generator_variability,
)

pd.options.display.max_columns = 200

/var/folders/nr/y3csklpd0l77v_k39y46s2rc0000gn/T/ipykernel_71125/2565956715.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


## Import settings and create database connections
This assumes that the settings file is set up for multiple scenarios/planning periods. If you are using a settings file with only a single scenario/planning period, remove or comment out the line with `build_scenario_settings`.

Once the settings file is loaded, use the `data_years` to limit the amount of data processed in `pudl_out`. Limiting the data years will reduce run time. 

Finally, check the settings for some common user errors. These currently include:
- Are all aggregated regions in `region_aggregations` valid IPM regions?
- Are all model regions included in the parameters `cost_multiplier_region_map` and `aeo_fuel_region_map`?
- Are any column names included more than once in `generator_columns`?
- The AEO reference scenario names for fuel cost and demand growth are of the form `REF<AEO year>`. Does the AEO year match the parameter `eia_aeo_year`?
- Are the technologies in `atb_new_gen` all valid names?

Most of these catch simple mistakes like misspelled region/technology names. If you would like any additional checks included, [submit an issue](https://github.com/PowerGenome/PowerGenome/issues) or make the changes yourself and submit a pull request.

In [2]:
cwd = Path.cwd()

settings_path = (
    cwd.parent / "example_systems" / "ISONE" / "settings"
)
settings = load_settings(settings_path)
settings["input_folder"] = settings_path.parent / settings["input_folder"]
scenario_definitions = pd.read_csv(
    settings["input_folder"] / settings["scenario_definitions_fn"]
)
scenario_settings = build_scenario_settings(settings, scenario_definitions)

pudl_engine, pudl_out, pg_engine = init_pudl_connection(
    freq="AS",
    start_year=min(settings.get("eia_data_years")),
    end_year=max(settings.get("eia_data_years")),
)

check_settings(settings, pg_engine)

The parameter value 'sampled' from column 'time_series' in your scenario definitions file is not included in the 'settings_management' dictionary. Settings for case id 's1' will not be modified to reflect this scenario.


## Initialize a `GeneratorClusters` object

`GeneratorClusters` is how existing generators are clustered, and it provides a convinience method for creating new-build resources (`GeneratorClusters.create_new_generators`).

In [3]:
gc = GeneratorClusters(pudl_engine, pudl_out, pg_engine, scenario_settings[2030]["p1"])

/Users/gs5183/miniconda3/envs/powergenome/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/gs5183/Documents/PowerGenome/powergenome/resource_clusters.py:274: FutureWarning: 'ParquetDataset.schema' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Specify 'use_legacy_dataset=False' while constructing the ParquetDataset, and then use the '.schema' attribute instead (which will return an Arrow schema instead of a Parquet schema).
  self._columns = self._dataset.schema.names


36.800000000000004  MW without lat/lon


### Existing generators

Lets look at how existing generators have been clustered in the different regions. The raw output from `create_region_technology_clusters` has lots of extra columns that might not be needed. Some of them include:

- `resource` is a snake case version of `technology`.
- `unmodified_cap_size` is the average size of generating units. `Cap_size` is `unmodified_cap_size` multiplied by the capacity factor for technologies that are derated by their CF.
- `Existing_Cap_MW` is `Cap_size` multiplied by `num_units`.
- `heat_rate_mmbtu_mwh_iqr` and `heat_rate_mmbtu_mwh_std` are measurements of how widely the heate rate varies within a cluster. If these values are large compared to `Heat_rate_MMBTU_per_MWh` then you might consider increasing the number of clusters for that technology/region.
- `Fuel` is a combination of the region (assigned by the settings parameter `fuel_region_map`), the AEO scenario (assigned by the settings parameter `aeo_fuel_scenarios`), and the fuel type for that generator resource (assigned by the settings parameters `tech_fuel_map` and `ccs_fuel_map`). Only fuels in AEO are included, which means that biomass/hydrogen/RNG, etc are not options.

The various "model tags" that are included in the settings file are unique to GenX. If you find them useful, they can be used to assign various values of any data type to generators in a column name of your choosing. If you don't want to use them, they can be ignored or removed via `settings.pop(<name>)`.

In [4]:
existing_gen = gc.create_region_technology_clusters()

********************************
When adding plant entity/boiler info to generators and filling missing seasonal capacity values, technologyNatural Gas Fired Combined Cycle changed capacity from 15950.7 to 16054.1
********************************
2023-06-13 06:57:35 [    INFO] catalystcoop.pudl.transform.eia861:456 Started with 325 missing BA Codes out of 12670 records (2.57%)
2023-06-13 06:57:35 [    INFO] catalystcoop.pudl.transform.eia861:480 Ended with 325 missing BA Codes out of 12670 records (2.57%)
2023-06-13 06:57:35 [    INFO] catalystcoop.pudl.output.eia860:177 97.4% of plant records have consistently reported BA Codes
2023-06-13 06:57:35 [    INFO] catalystcoop.pudl.output.eia860:227 Before any filling treatment has been applied. 2.6% of records have no BA codes
2023-06-13 06:57:35 [    INFO] catalystcoop.pudl.output.eia860:227 Backfilling and consistent value is the same. Filled w/ most consistent BA code. 2.6% of records have no BA codes
2023-06-13 06:57:35 [    INFO] cata

Creating gdf
['Other Natural Gas', 'Solar Photovoltaic', 'Batteries', 'Natural Gas Fired Combustion Turbine', 'Conventional Hydroelectric']
Creating gdf
['Other_peaker', 'Natural Gas Fired Combustion Turbine', 'Solar Photovoltaic', 'Other Natural Gas', 'Natural Gas Steam Turbine', 'Natural Gas Fired Combined Cycle', 'Conventional Hydroelectric', 'Batteries', 'Onshore Wind Turbine', 'Biomass', 'All Other']


No model tag values found for LDS ('LDS')
No model tag values found for Hydro_Energy_to_Power_Ratio ('Hydro_Energy_to_Power_Ratio')
No model tag values found for MinCapTag_2 ('MinCapTag_2')
No model tag values found for MinCapTag_3 ('MinCapTag_3')
No model tag values found for Reg_Max ('Reg_Max')
No model tag values found for Rsv_Max ('Rsv_Max')
/Users/gs5183/Documents/PowerGenome/powergenome/generators.py:3554: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.results["profile"][i] = clusters["profile"][0]
/Users/gs5183/Documents/PowerGenome/powergenome/generators.py:3554: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
existing_gen

,index,region,technology,cluster,Cap_Size,Existing_Cap_MWh,minimum_load_mw,Heat_Rate_MMBTU_per_MWh,Fixed_OM_Cost_per_MWyr,Var_OM_Cost_per_MWh,heat_rate_mmbtu_mwh_iqr,heat_rate_mmbtu_mwh_std,fixed_o_m_mw_std,Min_Power,num_units,plant_id_eia,unit_id_pg,capacity_factor,unmodified_cap_size,Existing_Cap_MW,profile,Start_Fuel_MMBTU_per_MW,Fuel,Start_Cost_per_MW,THERM,VRE,Num_VRE_Bins,MUST_RUN,STOR,FLEX,HYDRO,LDS,Commit,ESR_1,ESR_2,New_Build,Hydro_level,CapRes_1,CapRes_2,Hydro_Energy_to_Power_Ratio,MinCapTag_1,MinCapTag_2,MinCapTag_3,Reg_Max,Rsv_Max,Resource
0,0,NENGREST,Batteries,1,2.733,629.4,0.022,10.340,6235.000,0.000,0.000,0.000,0.000,0.008,103,"[1615, 1615, 1682, 60518, 60562, 60815, 60815,...","[1615_2, 1615_20, 1682_BS1, 60518_BA1, 60562_G...",0.135,2.733,281.499,None,0.0,None,0.000000,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.00,0.00,0,0,0,0,0,0,NENGREST_batteries_1
1,1,NENGREST,Biomass,1,2.651,0.0,4.960,15.591,150850.000,6.698,4.364,3.932,0.000,0.425,55,"[589, 2367, 10290, 10823, 10838, 10839, 50208,...","[1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 50365...",0.227,11.678,145.805,None,0.0,None,0.000000,0,0,0,1,0,0,0,0,1,1,1,-1,0.0,0.90,0.90,0,0,0,0,0,0,NENGREST_biomass_1
2,2,NENGREST,Conventional Hydroelectric,1,10.400,0.0,2.968,10.340,47048.000,0.000,0.000,0.000,0.000,0.285,66,"[1605, 1605, 1629, 1629, 1629, 1629, 1629, 162...","[1605_1, 1605_2, 1629_1, 1629_2, 1629_3, 1629_...",0.329,10.400,686.400,"[0.4467, 0.4465, 0.4464, 0.4462, 0.446, 0.4458...",0.0,None,0.000000,0,0,0,0,0,0,1,0,0,0,1,-1,0.5,0.80,0.80,0,0,0,0,0,0,NENGREST_conventional_hydroelectric_1
3,3,NENGREST,Conventional Steam Coal,1,133.650,0.0,56.500,14.421,78375.199,1.879,2.751,3.519,1125.174,0.423,4,"[2364, 2364, 2367, 2367]","[1.0, 2.0, 1.0, 3.0]",0.014,133.650,534.600,None,16.5,new_england_reference_coal,123.906288,1,0,0,0,0,0,0,0,1,0,0,0,0.0,0.90,0.90,0,0,0,0,0,0,NENGREST_conventional_steam_coal_1
4,4,NENGREST,Hydroelectric Pumped Storage,1,294.667,27404.0,156.667,10.340,47048.000,0.000,0.000,0.000,0.000,0.532,6,"[547, 547, 547, 547, 8005, 8005]","[547_1, 547_2, 547_3, 547_4, 8005_GEN1, 8005_G...",-0.026,294.667,1768.002,None,0.0,None,0.000000,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.95,0.95,0,0,0,0,0,0,NENGREST_hydroelectric_pumped_storage_1
5,5,NENGREST,Natural Gas Fired Combined Cycle,1,292.783,0.0,147.561,8.192,12410.908,3.979,1.377,1.355,3379.366,0.504,30,"[1595, 1660, 1660, 1682, 3236, 3236, 3236, 608...","[1.0, 1660_CC2, 1660_CC3, 2.0, 1.0, 2.0, 3.0, ...",0.140,292.783,8783.490,None,2.0,new_england_reference_naturalgas,92.081161,1,0,0,0,0,0,0,0,1,0,0,0,0.0,0.90,0.90,0,0,0,0,0,0,NENGREST_natural_gas_fired_combined_cycle_1
6,6,NENGREST,Natural Gas Fired Combustion Turbine,1,38.259,0.0,12.761,10.436,11358.833,5.279,0.943,1.126,1297.949,0.334,27,"[1599, 1660, 1660, 1678, 1678, 1678, 10108, 10...","[1599_3, 1660_WAT1, 1660_WAT2, 1678_1, 1678_2,...",0.105,38.259,1032.993,None,3.5,new_england_reference_naturalgas,119.396003,1,0,0,0,0,0,0,0,1,0,0,0,0.0,0.90,0.90,0,0,0,0,0,0,NENGREST_natural_gas_fired_combustion_turbine_1
7,7,NENGREST,Natural Gas Steam Turbine,1,23.340,0.0,3.150,15.120,51303.000,1.056,1.494,5.351,0.000,0.135,20,"[8002, 10417, 10883, 50041, 50041, 50087, 5008...","[1.0, 10417_TG, 1.0, 50041_GEN1, 50041_GEN2, 5...",0.315,23.340,466.800,None,13.7,new_england_reference_naturalgas,87.478829,1,0,0,0,0,0,0,0,1,0,0,0,0.0,0.90,0.90,0,0,0,0,0,0,NENGREST_natural_gas_steam_turbine_1
8,8,NENGREST,Nuclear,1,1251.400,0.0,360.000,10.446,275171.000,2.841,NaN,NaN,NaN,0.288,1,NaN,NaN,NaN,1251.400,1251.400,None,0.0,new_england_reference_uranium,247.835588,1,0,0,0,0,0,0,0,1,0,1,0,0.0,0.90,0.90,0,0,0,0,0,0,NENGREST_nuclear_1
9,9,NENGREST,Offshore Wind Turbine,1,29.300,0.0,0.100,10.340,82759.000,0.000,NaN,NaN,NaN,0.003,1,NaN,NaN,0.467,29.300,29.300,None,0.0,None,0.000000,0,1,1,0,0,0,0,0,0,0,0,0,0.0,0.00,0.00,0,0,0,0,0,0,NENGREST_offshore_wind_turbine_1


### New generators

New generators are based on data from NREL's Annual Technology Baseline (ATB). ATB uses a format of `\<technology>_<tech_detail>` to describe resources, such as `NaturalGas_CTAvgCF`. There are additional parameters including the cost case and financial case that are needed to specify the capex for a specific generator type. Note that many of the `tech_detail` values have identical capex and O&M values, so it doesn't matter which one is used. Examples include the capacity factor of combustion resources (`CTAvgCF` vs `CTHighCF`) and UtilityPV (`Chicago`, `KansasCity`, and `LosAngeles` have different capacity factors that we don't use in PowerGenome).

The raw output from `create_new_generators` has many more columns than existing generators. Several of these - such as `basis_year`, `capex`, `capex_mwh`, `cap_recovery_years`, `waccnomtech`, `regional_cost_multiplier`, and `interconnect_annuity` - are used to calculate the final `Inv_cost_per_MWyr` and `Inv_cost_per_MWhyr`. `lcoe` is pre-calculated using 2030 mid-range ATB costs, and is not specific to the model year for each run. The underlying data are retained here so they can be easily reviewed, but only keep the columns that you want/need.

The column `variability` has array values with the annual generation profiles for a resource. If these profiles represent 2012 they have 8784 hourly values.

The column `Max_Cap_MW` has a value of -1 if there is no limit on the capacity. 

If the user has included demand response profiles as an external file (using the `demand_response_fn` parameter in settings and the associated `demand_response` and `demand_response_resources` parameters), demand response resources will be included.

### Modified and custom new generators

By default, new generators are created using data from NREL ATB. But it is possible to modify an existing ATB generator type or create a modified copy of an ATB generator. To understand 

Modified ATB generators are included in the `atb_modifiers` parameter. In ATB 2019, NREL used a blend of combustion turbine and combined cycle types that we felt was unrealistic. Therefore we have included, by default, the following modifications. The modifying values are provided as `[<op>, <value>]`, where `op` is an operator of type `add`, `mul`, `sub`, and `truediv`. Again, this parameter modifies an existing technology in-place.

```
atb_modifiers:
  ngct:
    technology: NaturalGas
    tech_detail: CTAvgCF
    capex: [mul, 0.76]
    Var_OM_cost_per_MWh: [mul, 1.51]
    Fixed_OM_cost_per_MWyr: [mul, 0.56]
    Heat_rate_MMBTU_per_MWh: [mul, 0.97]
  ngcc:
    technology: NaturalGas
    tech_detail: CCAvgCF
    capex: [mul, 0.89]
    Var_OM_cost_per_MWh: [mul, 0.73]
    Fixed_OM_cost_per_MWyr: [mul, 0.95]
    Heat_rate_MMBTU_per_MWh: [mul, 0.98]
```

New generators based on modifications of an existing ATB generator are included in the parameter `modified_atb_new_gen`. The example below creates a NGCC resource called `NaturalGas_CCS100_Mid` with 100% CO₂ capture. Any new resources need to be added to the parameters `cost_multiplier_technology_map`, `new_build_startup_costs`, and `model_tag_values` (optional). Names are linked using string matching, so the full name doesn't need to be included in these parameters (but it should be long enough to be unique).

```
modified_atb_new_gen:
  NGCCS100:
    new_technology: NaturalGas
    new_tech_detail: CCS100
    new_cost_case: Mid
    atb_technology: NaturalGas
    atb_tech_detail: CCCCSAvgCF
    atb_cost_case: Mid
    size_mw: 500
    capex: [add, 116000]
    heat_rate: [add, 0.365]
    o_m_fixed_mw: [add, 9670]
    o_m_variable_mwh: [mul, 1.076]
```

### New wind/solar resources

Users can aggregate the ~500k potential wind and solar sites into a smaller number of representative clusters. The system is flexible, allowing the sites to be filtered, binned, grouped, and clustered according to different input features from the resource group data files. The method of aggregating can be specified by model region or applied to all model regions.

```yaml
renewables_clusters:
  - region: all
    technology: landbasedwind
    filter:
      - feature: lcoe
        max: 75
    bin:
      - feature: lcoe
        bins: [0, 40, 55, 76]
      - feature: cf
        q: 2
  - region: all
    technology: solar
    filter:
      - feature: lcoe
        max: 50
    bin:
      - feature: lcoe
        weights: mw
        q: 3
      - feature: cf
        q: 2
  - region: all
    technology: offshorewind
    turbine_type: fixed # options are fixed or floating
    pref_site: 1 # boolean variable for preferred sites (BOEM)
    filter:
      - feature: lcoe
        max: 125
    bin:
      - feature: lcoe
        weights: mw
        q: 3
    cluster:
      - feature: profile
        n_clusters: 2
        method: agg # Agglomerative clustering is the only method available
```


### Interconnection costs
Wind and solar resources have pre-calculated interconnection costs, so the user doesn't need to supply any additional data. But if a spur line is needed for thermal resources, these distances should be included under the column `spur_miles` in the file specified by `capacity_limit_spur_fn`. This spur line distance is then multiplied by costs for each model region listed in the `transmission_investment_cost` parameter. Spur line capex in the example settings file is from ReEDS documentation. You can look at [a mapping of IPM regions to ReEDS regions](https://github.com/gschivley/pg_misc/blob/master/create_clusters/site_interconnection_costs.py#L32-L155) and the associated transmission costs that I have compiled.

In [6]:
new_gen = gc.create_new_generators()

You have a renewables_cluster for technology 'solar in region 'NENGREST', but no comparable new-build technology was specified in your settings file.
You have a renewables_cluster for technology 'solar in region 'NENG_CT', but no comparable new-build technology was specified in your settings file.
You have a renewables_cluster for technology 'solar in region 'NENG_ME', but no comparable new-build technology was specified in your settings file.
/Users/gs5183/Documents/PowerGenome/powergenome/external_data.py:146: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace("_\*", "_all")
Transmission investment costs are missing or zero for some resources and will not be included in the total investment costs.
No model tag values found for LDS ('LDS')
No model tag values found for Hydro_Energy_to_Power_Ratio ('Hydro_Energy_to_Power_Ratio')
No model tag values found for MinCapTag_2 ('MinCapTag_2')
No model tag values found for MinCapTag_3 (

In [11]:
new_gen.columns

Index(['technology', 'basis_year', 'Fixed_OM_Cost_per_MWyr',
       'Fixed_OM_Cost_per_MWhyr', 'Var_OM_Cost_per_MWh', 'capex_mw',
       'capex_mwh', 'Inv_Cost_per_MWyr', 'Inv_Cost_per_MWhyr',
       'Heat_Rate_MMBTU_per_MWh',
       ...
       'Flexible_Demand_Energy_Eff', 'Ramp_Up_Percentage',
       'Ramp_Dn_Percentage', 'Up_Time', 'Down_Time', 'NACC_Eff',
       'NACC_Peak_to_Base', 'Reg_Cost', 'Rsv_Cost', 'Resource'],
      dtype='object', length=106)

#### Spur line/interconnection distances and capacity limits
Interconnection distances and the maximum available capacity for UtilityPV, LandbasedWind, and OffshoreWind are all included in the data used to select and combine clusters. Spur line distances for other resources must be provided by the user in a CSV file and included in the settings file as `capacity_limit_spur_fn`.

In [9]:
cols = [
    "region",
    "technology",
    "cluster",
    "Max_Cap_MW",
    "lcoe",
    "capex_mw",
    "regional_cost_multiplier",
    "Inv_Cost_per_MWyr",
    "plant_inv_cost_mwyr",
    "Start_Cost_per_MW",
    "interconnect_annuity",
    "spur_inv_mwyr",
    "spur_miles",
    "profile",
]
new_gen[cols]

,region,technology,cluster,Max_Cap_MW,lcoe,capex_mw,regional_cost_multiplier,Inv_Cost_per_MWyr,plant_inv_cost_mwyr,Start_Cost_per_MW,interconnect_annuity,spur_inv_mwyr,spur_miles,profile
0,NENGREST,NaturalGas_CCCCSAvgCF_Conservative,0,-1.000000,0.000000,2.405433e+06,1.061892,193825.048927,185324.0,105.947023,0.000000,8501.048927,20,0
1,NENGREST,NaturalGas_CCAvgCF_Moderate,0,-1.000000,0.000000,9.552156e+05,1.185535,100728.000000,100728.0,105.947023,0.000000,0.000000,0,0
2,NENGREST,NaturalGas_CTAvgCF_Moderate,0,-1.000000,0.000000,8.340070e+05,1.128169,83691.000000,83691.0,137.375017,0.000000,0.000000,0,0
3,NENGREST,Battery_*_Moderate,0,-1.000000,0.000000,2.485542e+05,1.030369,20761.000000,20761.0,0.000000,0.000000,0.000000,0,0
4,NENGREST,LandbasedWind_Class3_Moderate_,1,350.358747,40.126613,1.207730e+06,1.266111,123763.631264,107181.0,0.000000,16582.631264,0.000000,0,"[0.3265099, 0.26280975, 0.3775145, 0.33958435,..."
5,NENGREST,LandbasedWind_Class3_Moderate_,2,316.271250,36.732337,1.207730e+06,1.266111,134124.217757,107181.0,0.000000,26943.217757,0.000000,0,"[0.76192015, 0.67903197, 0.8000185, 0.7520272,..."
6,NENGREST,LandbasedWind_Class3_Moderate_,3,175.972494,45.329121,1.207730e+06,1.266111,153500.627089,107181.0,0.000000,46319.627089,0.000000,0,"[0.79786664, 0.79143715, 0.8120507, 0.7896965,..."
7,NENGREST,LandbasedWind_Class3_Moderate_,4,517.994993,45.312804,1.207730e+06,1.266111,170301.839866,107181.0,0.000000,63120.839866,0.000000,0,"[0.99999994, 0.99999994, 0.8464283, 0.87262315..."
8,NENGREST,LandbasedWind_Class3_Moderate_,5,72.495000,52.981000,1.207730e+06,1.266111,137976.848773,107181.0,0.000000,30795.848773,0.000000,0,"[0.1497182, 0.11176389, 0.14328389, 0.12527987..."
9,NENGREST,LandbasedWind_Class3_Moderate_,6,534.667508,47.035431,1.207730e+06,1.266111,176112.447869,107181.0,0.000000,68931.447869,0.000000,0,"[0.88626117, 0.8320974, 0.5167011, 0.5944716, ..."


#### Generation profiles

Hourly generation profiles are saved in a `variability` column of the dataframe. These are then extracted using the function `make_generator_variability`. The variability (generation profile) dataframe is in the same (column) order as rows in the generator dataframe.

In [10]:
existing_variability = make_generator_variability(existing_gen)
existing_variability

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
0,1.0,1.0,0.4467,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.2949,1.0,0.4467,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.1727,1.0,0.3409,0.0,1.0,1.0,0.6644,1.0,1.0,1.0,0.5246,1.0,0.6644,0.0
1,1.0,1.0,0.4465,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.2855,1.0,0.4465,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.1644,1.0,0.3409,0.0,1.0,1.0,0.6646,1.0,1.0,1.0,0.5049,1.0,0.6646,0.0
2,1.0,1.0,0.4464,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.2822,1.0,0.4464,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.2645,1.0,0.3409,0.0,1.0,1.0,0.6647,1.0,1.0,1.0,0.4734,1.0,0.6647,0.0
3,1.0,1.0,0.4462,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.2999,1.0,0.4462,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.2140,1.0,0.3409,0.0,1.0,1.0,0.6648,1.0,1.0,1.0,0.4430,1.0,0.6648,0.0
4,1.0,1.0,0.4460,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.2623,1.0,0.4460,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.1407,1.0,0.3409,0.0,1.0,1.0,0.6649,1.0,1.0,1.0,0.4305,1.0,0.6649,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,1.0,1.0,0.4476,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.1702,1.0,0.4476,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.1340,1.0,0.3409,0.0,1.0,1.0,0.6638,1.0,1.0,1.0,0.2243,1.0,0.6638,0.0
8756,1.0,1.0,0.4474,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.1729,1.0,0.4474,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.1134,1.0,0.3409,0.0,1.0,1.0,0.6639,1.0,1.0,1.0,0.1976,1.0,0.6639,0.0
8757,1.0,1.0,0.4472,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.1893,1.0,0.4472,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.1329,1.0,0.3409,0.0,1.0,1.0,0.6641,1.0,1.0,1.0,0.2626,1.0,0.6641,0.0
8758,1.0,1.0,0.4471,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.2851,1.0,0.4471,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.2760,1.0,0.3409,0.0,1.0,1.0,0.6642,1.0,1.0,1.0,0.2805,1.0,0.6642,0.0


Since the variability column names are only integers, it can help to replace them with descriptive strings.

In [11]:
existing_variability.columns = (
    existing_gen["region"]
    + "_"
    + existing_gen["Resource"]
    + "_"
    + existing_gen["cluster"].astype(str)
)
existing_variability

,NENGREST_NENGREST_batteries_1_1,NENGREST_NENGREST_biomass_1_1,NENGREST_NENGREST_conventional_hydroelectric_1_1,NENGREST_NENGREST_conventional_steam_coal_1_1,NENGREST_NENGREST_hydroelectric_pumped_storage_1_1,NENGREST_NENGREST_natural_gas_fired_combined_cycle_1_1,NENGREST_NENGREST_natural_gas_fired_combustion_turbine_1_1,NENGREST_NENGREST_natural_gas_steam_turbine_1_1,NENGREST_NENGREST_nuclear_1_1,NENGREST_NENGREST_offshore_wind_turbine_1_1,NENGREST_NENGREST_onshore_wind_turbine_1_1,NENGREST_NENGREST_other_peaker_1_1,NENGREST_NENGREST_small_hydroelectric_1_1,NENGREST_NENGREST_solar_photovoltaic_1_1,NENG_CT_NENG_CT_batteries_1_1,NENG_CT_NENG_CT_biomass_1_1,NENG_CT_NENG_CT_conventional_hydroelectric_1_1,NENG_CT_NENG_CT_hydroelectric_pumped_storage_1_1,NENG_CT_NENG_CT_natural_gas_fired_combined_cycle_1_1,NENG_CT_NENG_CT_natural_gas_fired_combustion_turbine_1_1,NENG_CT_NENG_CT_natural_gas_steam_turbine_1_1,NENG_CT_NENG_CT_nuclear_1_1,NENG_CT_NENG_CT_onshore_wind_turbine_1_1,NENG_CT_NENG_CT_other_peaker_1_1,NENG_CT_NENG_CT_small_hydroelectric_1_1,NENG_CT_NENG_CT_solar_photovoltaic_1_1,NENG_ME_NENG_ME_batteries_1_1,NENG_ME_NENG_ME_biomass_1_1,NENG_ME_NENG_ME_conventional_hydroelectric_1_1,NENG_ME_NENG_ME_natural_gas_fired_combined_cycle_1_1,NENG_ME_NENG_ME_natural_gas_fired_combustion_turbine_1_1,NENG_ME_NENG_ME_natural_gas_steam_turbine_1_1,NENG_ME_NENG_ME_onshore_wind_turbine_1_1,NENG_ME_NENG_ME_other_peaker_1_1,NENG_ME_NENG_ME_small_hydroelectric_1_1,NENG_ME_NENG_ME_solar_photovoltaic_1_1
0,1.0,1.0,0.4467,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.2949,1.0,0.4467,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.1727,1.0,0.3409,0.0,1.0,1.0,0.6644,1.0,1.0,1.0,0.5246,1.0,0.6644,0.0
1,1.0,1.0,0.4465,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.2855,1.0,0.4465,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.1644,1.0,0.3409,0.0,1.0,1.0,0.6646,1.0,1.0,1.0,0.5049,1.0,0.6646,0.0
2,1.0,1.0,0.4464,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.2822,1.0,0.4464,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.2645,1.0,0.3409,0.0,1.0,1.0,0.6647,1.0,1.0,1.0,0.4734,1.0,0.6647,0.0
3,1.0,1.0,0.4462,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.2999,1.0,0.4462,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.2140,1.0,0.3409,0.0,1.0,1.0,0.6648,1.0,1.0,1.0,0.4430,1.0,0.6648,0.0
4,1.0,1.0,0.4460,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.2623,1.0,0.4460,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.1407,1.0,0.3409,0.0,1.0,1.0,0.6649,1.0,1.0,1.0,0.4305,1.0,0.6649,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,1.0,1.0,0.4476,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.1702,1.0,0.4476,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.1340,1.0,0.3409,0.0,1.0,1.0,0.6638,1.0,1.0,1.0,0.2243,1.0,0.6638,0.0
8756,1.0,1.0,0.4474,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.1729,1.0,0.4474,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.1134,1.0,0.3409,0.0,1.0,1.0,0.6639,1.0,1.0,1.0,0.1976,1.0,0.6639,0.0
8757,1.0,1.0,0.4472,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.1893,1.0,0.4472,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.1329,1.0,0.3409,0.0,1.0,1.0,0.6641,1.0,1.0,1.0,0.2626,1.0,0.6641,0.0
8758,1.0,1.0,0.4471,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.2851,1.0,0.4471,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.2760,1.0,0.3409,0.0,1.0,1.0,0.6642,1.0,1.0,1.0,0.2805,1.0,0.6642,0.0


In [12]:
make_generator_variability(new_gen)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
0,1.0,1.0,1.0,1.0,0.326510,0.761920,0.797867,1.000000,0.149718,0.886261,0.448254,0.814466,0.694686,0.892695,0.590493,0.513469,1.0,1.0,1.0,1.0,0.848894,0.870309,0.630747,0.585088,0.293596,0.512011,1.0,1.0,1.0,1.0,0.922187,0.956431,0.919378,0.936693,0.717576,0.820905,0.112689,0.104027,0.126638,0.317829,0.316278,0.296490
1,1.0,1.0,1.0,1.0,0.262810,0.679032,0.791437,1.000000,0.111764,0.832097,0.510635,0.803070,0.726342,0.866608,0.660334,0.335984,1.0,1.0,1.0,1.0,0.858507,0.829902,0.600321,0.560315,0.266590,0.453836,1.0,1.0,1.0,1.0,0.895880,0.926753,0.862824,0.920390,0.654849,0.769258,0.043916,0.040452,0.049787,0.194500,0.192879,0.182809
2,1.0,1.0,1.0,1.0,0.377515,0.800018,0.812051,0.846428,0.143284,0.516701,0.420613,0.637623,0.566849,0.702615,0.520600,0.182269,1.0,1.0,1.0,1.0,0.914197,0.995875,0.800432,0.756769,0.355582,0.634766,1.0,1.0,1.0,1.0,0.894452,0.893841,0.859792,0.899866,0.630715,0.745312,0.035556,0.032538,0.040722,0.082102,0.080851,0.077988
3,1.0,1.0,1.0,1.0,0.339584,0.752027,0.789697,0.872623,0.125280,0.594472,0.393500,0.478903,0.434019,0.529905,0.374286,0.177643,1.0,1.0,1.0,1.0,0.880523,0.897995,0.710771,0.650341,0.295549,0.623322,1.0,1.0,1.0,1.0,0.865521,0.880251,0.820155,0.898597,0.562186,0.716275,0.024355,0.022301,0.028550,0.090504,0.089019,0.087735
4,1.0,1.0,1.0,1.0,0.246250,0.641637,0.760396,0.917902,0.071759,0.600711,0.359101,0.558137,0.482576,0.618053,0.358666,0.263142,1.0,1.0,1.0,1.0,0.677017,0.633942,0.435773,0.432344,0.191068,0.391429,1.0,1.0,1.0,1.0,0.827164,0.822829,0.774091,0.860527,0.500022,0.673711,0.004485,0.004101,0.005387,0.066136,0.064974,0.065513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,1.0,1.0,1.0,1.0,0.228918,0.401395,0.225927,0.143577,0.092088,0.055788,0.190230,0.310401,0.115799,0.414936,0.311953,0.972474,1.0,1.0,1.0,1.0,0.487207,0.793695,0.424499,0.387240,0.228279,0.271431,1.0,1.0,1.0,1.0,0.687864,0.591491,0.598528,0.356457,0.463492,0.307882,0.637675,0.584121,0.759315,0.632499,0.630755,0.633374
8756,1.0,1.0,1.0,1.0,0.204692,0.407841,0.333962,0.455153,0.051496,0.325811,0.344991,0.261895,0.207955,0.314317,0.317417,1.000000,1.0,1.0,1.0,1.0,0.387605,0.679635,0.386457,0.342486,0.165261,0.194961,1.0,1.0,1.0,1.0,0.682108,0.564835,0.566073,0.487513,0.373070,0.344967,0.539027,0.495542,0.637830,0.596518,0.595974,0.592530
8757,1.0,1.0,1.0,1.0,0.247542,0.506351,0.371844,0.493415,0.074760,0.359396,0.512571,0.379048,0.518319,0.318074,0.416731,1.000000,1.0,1.0,1.0,1.0,0.470877,0.676457,0.408538,0.367494,0.204503,0.279661,1.0,1.0,1.0,1.0,0.799232,0.692519,0.727896,0.619756,0.518921,0.474596,0.451404,0.415318,0.535608,0.559930,0.559392,0.557209
8758,1.0,1.0,1.0,1.0,0.325462,0.580577,0.392696,0.530781,0.113237,0.401569,0.644958,0.647941,0.771556,0.561428,0.554013,0.998759,1.0,1.0,1.0,1.0,0.695259,0.786558,0.592772,0.569617,0.229682,0.369646,1.0,1.0,1.0,1.0,0.836833,0.752635,0.745671,0.696048,0.519881,0.507354,0.354310,0.324257,0.413795,0.388990,0.386778,0.381227
